In [1]:
import os
import time
import datetime
import pandas as pd
import tweepy as tw
import numpy as np
consumer_key ='w1DxpyxLyTuDSfVReQqg4dzKG'
consumer_secret= 'JHfiE2JdjzL9CwTkGU5JQPfNhT8mNQnTDWaq9Ln37zFI2YyU1p'
access_token= '2614749122-HPp0Rb2mZ5BP6EYxdnnAKlSvU9lJytLo14g4Hcx'
access_token_secret= 'OkPXAohHe4XJ2JPX6YGfwteTCGu5vB7V51iiVa2Cjphhh'
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True) #rate limit is 200 requests every 15 minutes
training_data = pd.read_csv('botometer-feedback-2019-data.tsv',sep = '\t',names = ['ID','labels'])   #repository data
training_data = training_data.replace({'human':0, 'bot': 1}).to_numpy()  #529 accounts
print(training_data.shape)
labels = training_data[:,1]
current_time = datetime.datetime.now()
users_features = np.empty([0,8])
users_labels = np.empty([0,1])
for i in range(529):
    try:
        user_id = (training_data[i,0])
        user = api.get_user(user_id)
        user_features = (user.followers_count,
            user.friends_count,  #friends = following
            user.listed_count,   #number of lists/groups member of
           user.favourites_count,  #tweets liked in lifetime
           user.statuses_count, #tweets/retweets in lifetime 
           int(user.default_profile), #default profile background True/False (1/0)
           int(user.default_profile_image), #default profile image True/False (1/0)
           ((current_time-user.created_at).days))   #account age (days, rounded down), for avg tweets/day
        user_features = np.array(user_features).reshape((1,8))
        users_features = np.concatenate([users_features,user_features])
        users_labels = np.append(users_labels,labels[i])
    except tw.error.TweepError:
        print("user not found")
        

print(users_features.shape)
print(users_features)
print(users_labels.shape)



(529, 2)


KeyboardInterrupt: 

In [2]:
import os
import time
import datetime
import pandas as pd
import tweepy as tw
import numpy as np
consumer_key ='w1DxpyxLyTuDSfVReQqg4dzKG'
consumer_secret= 'JHfiE2JdjzL9CwTkGU5JQPfNhT8mNQnTDWaq9Ln37zFI2YyU1p'
access_token= '2614749122-HPp0Rb2mZ5BP6EYxdnnAKlSvU9lJytLo14g4Hcx'
access_token_secret= 'OkPXAohHe4XJ2JPX6YGfwteTCGu5vB7V51iiVa2Cjphhh'
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True) #rate limit is 200 requests every 15 minutes
training_data = pd.read_csv('botometer-feedback-2019-data.tsv',sep = '\t',names = ['ID','labels'])   #repository data
training_data = training_data.replace({'human':0, 'bot': 1}).to_numpy() 
print(training_data.shape)

(529, 2)


In [3]:
def follower_lessthan50(user_id): # returns 1 if less than 20 followers
    user = api.get_user(user_id)
    if int(user.followers_count) < 20:
        return 1
    else:
        return 0
def default_description(user_id): #returns 1 if no description
    user = api.get_user(user_id)
    if user.description == '':
        return 1
    else: 
        return 0
def link_in_description(user_id): #returns 1 if http in  description
    user = api.get_user(user_id)
    if 'http' in user.description:
        return 1
    else: 
        return 0
def default_location(user_id):  #returns 1 if no location listed
    user = api.get_user(user_id)
    if user.location == '':
        return 1
    else: 
        return 0
def ftf_ratio50(user_id):    #returns 1 if following:follower ratio is >50
    user = api.get_user(user_id)
    if int(user.friends_count) > 50*int(user.followers_count):
        return 1
    else: 
        return 0    
def ftf_ratio100(user_id):    #returns 1 if following:follower ratio is >100
    user = api.get_user(user_id)
    if int(user.friends_count) > 100*int(user.followers_count):
        return 1
    else: 
        return 0 
def tweet_lessthan50(user_id):    #1 if less than 50 tweets
    user = api.get_user(user_id)
    if int(user.statuses_count) < 50:
        return 1
    else: 
        return 0
def tweet_morethan1000(user_id):   #1 if more than 1000 tweets
    user = api.get_user(user_id)
    if int(user.statuses_count) > 1000:
        return 1
    else: 
        return 0 
training_data = pd.read_csv('botometer-feedback-2019-data.tsv',sep = '\t',names = ['ID','labels'])   #repository data
training_data = training_data.replace({'human':0, 'bot': 1}).to_numpy()  #529 accounts
print(training_data.shape)
labels = training_data[:,1]
current_time = datetime.datetime.now()
users_features = np.empty([0,10])
users_labels = np.empty([0,1])
for i in range(training_data.shape[0]): 
    try:    
        user_id = (training_data[i,0])
        user = api.get_user(user_id)
        user_features = (follower_lessthan50(user_id),
            ftf_ratio50(user_id),
            ftf_ratio100(user_id),
           tweet_lessthan50(user_id),
            tweet_morethan1000(user_id),
           int(user.default_profile),
           int(user.default_profile_image),
                 default_description(user_id),
                 link_in_description(user_id), 
                default_location(user_id)) 
        user_features = np.array(user_features).reshape((1,10))
        users_features = np.concatenate([users_features,user_features])
        users_labels = np.append(users_labels,labels[i])
    except tw.error.TweepError:
        print("user not found")
#print(user)


#conversion to binary:
# location listd, if has description, if has a url listed, following:follower ratio, less than 50 followers
# more than 1000 followers, has sent more than x tweets, has sent less than x tweets
# if description contains a link

(529, 2)


KeyboardInterrupt: 